# Script to write MEOP data

Data prepared for Viktor Gouretski. July 2020.

* variable 1: profile ID#
* variable 2: Lat
* variable 3: Lon
* variable 4: Year
* variable 5: Month
* variable 6: Day
* variable 7: hour
* variable 8: pressure
* variable 9: temperature
* variable 10: salinity
* variable 11: validation flag for T
* variable 12: validation flag for S



In [2]:
from pathlib import Path
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gsw
import csv

import sys
sys.path.insert(0, '/home/jupyter-froqu/MEOP_analyses/')
import meop

In [3]:
def profile_data(ds):
    
    # ds: xarray dataset containing data from a tag
    ds['profileID']=(('N_PROF'),  ['{}_{:04d}'.format(ds.smru_platform_code,kk) for kk in range(ds.dims['N_PROF'])])
    ds['YEAR'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    ds['MONTH'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    ds['DAY'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    ds['HOUR'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    ds['MINUTE'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    ds['SECOND'] = (('N_PROF'),[0]*ds.dims['N_PROF'])
    for kk,time in enumerate(ds['JULD'].data):
        ds['YEAR'][kk]=time.year
        ds['MONTH'][kk]=time.month
        ds['DAY'][kk]=time.day
        ds['HOUR'][kk]=time.hour
        ds['MINUTE'][kk]=time.minute
        ds['SECOND'][kk]=time.second
    
    fieldnames = ['profileID','LATITUDE','LONGITUDE','YEAR','MONTH','DAY','HOUR','PRES_ADJUSTED','TEMP_ADJUSTED','PSAL_ADJUSTED','TEMP_QC','PSAL_QC']
    df = ds[fieldnames].to_dataframe()
    df = df.reset_index()
    df = df.set_index(['N_PROF', 'N_LEVELS']).sort_index()

    # remove rows with no data
    I = np.logical_or(df['TEMP_QC']==b'1', df['PSAL_QC']==b'1')
    df = df[I]

    # remove rows with no PRES data
    df = df[~np.isnan(df['PRES_ADJUSTED'])]
    
    # replace nan with 99.999
    df = df.fillna(99.999)
    
    df['PRES_ADJUSTED']=df['PRES_ADJUSTED'].astype(int)
    df['TEMP_QC']=df['TEMP_QC'].astype(int)
    df['PSAL_QC']=df['PSAL_QC'].astype(int)
    
    return df


In [4]:
def write_tag_csv(smru_name,folder='',qf='hr1'):
    
    depl = smru_name.split("-")[0]    
    ds = meop.read_ncfile(depl,smru_name,qf)
    df = profile_data(ds)
    if df.empty:
        return df
    df.loc[df.duplicated(subset='profileID',keep='first'),['profileID','LATITUDE','LONGITUDE','YEAR','MONTH','DAY','HOUR']]=np.nan
    filename = Path(folder,smru_name+'_'+qf+'.csv.gz')
    df.to_csv(filename, mode='w', index=False, float_format='%6.3f',compression='gzip')
    return df

In [7]:
df_all=meop.read_list_data_in_MEOP("list_meop.pkl")
df_public = meop.filter_public_data(df_all)
df_public

JULD  LATITUDE  LONGITUDE  \
SMRU_PLATFORM_CODE CYCLE_NUMBER                                             
awru1-A-06         1             2007-03-03 03:24:00  -68.5139    78.3736   
                   2             2007-03-03 03:43:00  -68.5122    78.3797   
                   3             2007-03-03 06:01:00  -68.4999    78.4232   
                   4             2007-03-03 17:41:00  -68.4860    78.5198   
                   5             2007-03-03 17:52:00  -68.4899    78.5180   
...                                              ...       ...        ...   
wd3-CTD3-07        420           2007-09-07 09:20:00  -66.5129   141.0960   
                   421           2007-09-08 03:20:00  -66.5261   141.0760   
                   422           2007-09-08 03:20:00  -66.5102   141.0760   
                   423           2007-09-13 17:10:00  -66.5102   141.0760   
                   424           2007-09-13 17:10:00  -66.5289   140.9110   

                                 N_TEMP  N_PSAL  N_CHLA  
SMRU_PLATFORM_CODE CYCLE_NUMBER                          
awru1-A-06         1                 12       0     0.0  
                   2                 12       0     0.0  
                   3                 12       0     0.0  
                   4                 12       0     0.0  
                   5                 12       0     0.0  
...                                 ...     ...     ...  
wd3-CTD3-07        420               10      10     0.0  
                   421               10      10     0.0  
                   422               10      10     0.0  
                   423               19      19     0.0  
                   424               19      19     0.0  

[644479 rows x 6 columns]

In [74]:
with open('list_tags_processed.txt', 'w') as log:
    log.write('tag, Nrows_hr\n')
        
for smru_name in df_public.index.levels[0]:
    depl = smru_name.split("-")[0]
    print(smru_name)
    try:
        df = write_tag_csv(smru_name,folder='MEOP_csv_gz',qf='hr1')
        flag_hr = df.shape[0]
    except FileNotFoundError:
        flag_hr = -1

    with open('list_tags_processed.txt', 'a') as log:
        log.write("{}, {}\n".format(smru_name,flag_hr))

awru1-A-06
awru1-B-06
awru1-C-06
bs1-53127-04
bs1-53128-04
bs1-53129-04
bs1-53130-04
bs2-53127-05
bs2-53128-05
bs2-53130-05
bv3-240-14
bv3-241-14


KeyboardInterrupt: 